
## Node2op
---
#### issue
* 현재 노드는 모두 동일하므로 노드가 어떤 opearation을 나타내는지 정보를 담을 공간이 없다
* graph 생성 -> graph 저장 -> 노드 DAG 변환 과정을 거쳤다면
* graph 생성 -> graph 끊겨 있는지 확인(connext_start2end) -> 노드 DAG 변환 + operation (node2op) 
* 임의의 그래프 생성해서 실행해보자
* 생각해보니 그래프를 저장할 필요 없음, 초기값만 생성하고 chromosome 을 저장하자!

In [1]:
import os
import sys
import random
import pprint


sys.path.insert(0,'../')
from utils_kyy.utils_graph import make_random_graph
from utils_kyy.utils_graph import *

import networkx as nx
import numpy as np
from easydict import EasyDict


In [2]:
def make_random_graph_ex(num_graph):

    # 해당 stage_pool_path에 stage가 꽉 차있는지 확인


        Nodes = random.randint(20, 40)  # => [Nodes 값 수정 시 주의] 아래 K값은 Nodes보다 작아야함.
        graph_model = 'WS'
        K = random.randint(4, Nodes-10)  # [min, max] // WS에서는 K nearest. 따라서, 4 ~ 30 random 선택 하도록
        P = round(random.uniform(0.25, 0.75), 2)   # 소수 둘째자리까지만 나오도록 반올림 => 결과 e.g. 0.75

        args = EasyDict({'graph_model': graph_model, 'K':K, 'P':P})

        P_str = str(P)[0] + str(P)[2:]   # 0.75 => 075

        graph = build_graph(Nodes, args)

        return graph


def get_node_info(graph):
    input_nodes = []
    output_nodes = []
    Nodes = []
    for node in range(graph.number_of_nodes()):
        # node i 에 대해        
        tmp = list(graph.neighbors(node))
        tmp.sort()    # 오름차순 정렬
    
        # node type 정의    
        type = -1    # input node도, output node도 아닌. 그래프의 중간에 매개자처럼 있는 중간 node.
        if node < tmp[0]:
            input_nodes.append(node)
            type = 0    # id 가장 작은 노드보다 작으면, 이건 외부에서 input을 받는 노드. 즉 input node.
        if node > tmp[-1]:
            output_nodes.append(node)
            type = 1    # id 가장 큰 노드보다 크면, 이건 외부로 output 내보내는 노드. 즉 output node.
        
        # dag로 변환 (자신의 id보다 작은 노드들과의 연결만 남기기)
        # [type] 0: input node, 1: output node, -1: input도 output도 아닌, 그래프 중간에 매개자처럼 있는 중간 node
        Nodes.append(Node(node, [n for n in tmp if n < node], type))    # DAG(Directed Acyclic Graph)로 변환
    return Nodes, input_nodes, output_nodes

In [3]:
grph_ex =make_random_graph_ex(1)

In [4]:
input_nodes = []
output_nodes = []
Nodes = []
for node in range(grph_ex.number_of_nodes()):
    
    ## 초기에 임의로 operation 넣어주자
    
    tmp = list(grph_ex.neighbors(node))
    tmp.sort()    # 오름차순 정렬
    # node type 정의    
    type = -1    # input node도, output node도 아닌. 그래프의 중간에 매개자처럼 있는 중간 node.
    if node < tmp[0]:
        input_nodes.append(node)
        type = 0    # id 가장 작은 노드보다 작으면, 이건 외부에서 input을 받는 노드. 즉 input node.
    if node > tmp[-1]:
        output_nodes.append(node)
        type = 1    # id 가장 큰 노드보다 크면, 이건 외부로 output 내보내는 노드. 즉 output node.

    # dag로 변환 (자신의 id보다 작은 노드들과의 연결만 남기기)
    # [type] 0: input node, 1: output node, -1: input도 output도 아닌, 그래프 중간에 매개자처럼 있는 중간 node
    Nodes.append(Node(node, [n for n in tmp if n < node], type))    # DAG(Directed Acyclic Graph)로 변환



In [5]:
from random import randint
randint(0,7)

5

#### check_connection - get grph-info 안에 넣어주자
* 처음과 끝 연결되어 있는지 확인
* 만약 끊겨 있으면 강제로 처음과 끝 연결

#### get_g_matrix
* edge에 operation 값(0,7)을 넣어준다
* adjanacy matrix 구하기 - 기본적으로 모두 7로 initialize (모두 연결이 안됨)

* edge 별로 주지 말고 Node 별로 주자 실험 1번은!!


In [6]:
def get_graph_info(graph):
    input_nodes = []
    output_nodes = []
    Nodes = []
    for node in range(graph.number_of_nodes()):
        # node i 에 대해        
        tmp = list(graph.neighbors(node))
        tmp.sort()    # 오름차순 정렬
    
        # node type 정의    
        type = -1    # input node도, output node도 아닌. 그래프의 중간에 매개자처럼 있는 중간 node.
        if node < tmp[0]:
            input_nodes.append(node)
            type = 0    # id 가장 작은 노드보다 작으면, 이건 외부에서 input을 받는 노드. 즉 input node.
        if node > tmp[-1]:
            output_nodes.append(node)
            type = 1    # id 가장 큰 노드보다 크면, 이건 외부로 output 내보내는 노드. 즉 output node.
        
        # dag로 변환 (자신의 id보다 작은 노드들과의 연결만 남기기)
        # [type] 0: input node, 1: output node, -1: input도 output도 아닌, 그래프 중간에 매개자처럼 있는 중간 node
        Nodes.append(Node(node, [n for n in tmp if n < node], type))    # DAG(Directed Acyclic Graph)로 변환
        
        
    ## Checking Connection
    connection_flag = False
    here =len(Nodes) -1 
    visited = [False] *(len(Nodes) -1)
    q = []
    neigbors = Nodes[here][1]

    q.append(neigbors[0])
    visited[neigbors[0]] = True


    while q :
  #      print("here",here)
        neigbors = Nodes[here][1]
 #       print("neigbors",neigbors)

        for n in neigbors:
            if visited[n] == False:
                visited[n] = True
                q.append(n)


        here = q[0]
        q = q[1:]

        if visited[0] == True:
#            print("Connected")
            connection_flag = True
            break


    if connection_flag == False :
        ## 강제로 연결
        Nodes[-1][1].insert(0,0)
        Nodes[-1][1] = sorted(Nodes[-1][1])
    
    return Nodes, input_nodes, output_nodes


def get_g_matrix(Nodes):
    nsize = len(Nodes)
    adj_mat = np.array([[7]*nsize for i in range(nsize)])
    random.seed(1234)
    

    for n in Nodes :
        node_id = n[0]
        for n_input in n[1]:
            operation = random.randint(0,6)
            adj_mat[n_input][node_id] = operation

    return adj_mat

def get_operation(Nodes):
    nsize = len(Nodes)
    oplist

In [7]:
## G matrix 구하기
random.seed(1234)
result = get_graph_info(grph_ex)
nds = result[0]

In [8]:
pprint.pprint(nds)

[Node(id=0, inputs=[], type=0),
 Node(id=1, inputs=[0], type=-1),
 Node(id=2, inputs=[0, 1], type=-1),
 Node(id=3, inputs=[1, 2], type=-1),
 Node(id=4, inputs=[0, 1, 2, 3], type=-1),
 Node(id=5, inputs=[0, 1, 2, 3, 4], type=-1),
 Node(id=6, inputs=[0, 1, 2, 3, 4, 5], type=-1),
 Node(id=7, inputs=[0, 1, 2, 3, 6], type=-1),
 Node(id=8, inputs=[0, 1, 2, 3, 4, 6, 7], type=-1),
 Node(id=9, inputs=[4, 5, 6, 8], type=-1),
 Node(id=10, inputs=[0, 3, 4, 5, 7, 8, 9], type=-1),
 Node(id=11, inputs=[1, 3, 4, 6, 7, 8, 9, 10], type=-1),
 Node(id=12, inputs=[0, 1, 3, 4, 5, 6, 10, 11], type=-1),
 Node(id=13, inputs=[1, 2, 7, 9, 10, 11, 12], type=-1),
 Node(id=14, inputs=[1, 5, 6, 7, 11, 13], type=-1),
 Node(id=15, inputs=[0, 1, 2, 3, 4, 5, 8, 9, 10, 12, 13, 14], type=-1),
 Node(id=16, inputs=[0, 1, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15], type=-1),
 Node(id=17, inputs=[0, 2, 3, 4, 5, 6, 9, 10, 11, 12, 14, 16], type=-1),
 Node(id=18, inputs=[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17], typ

In [19]:
nsize = len(nds)

In [10]:
g_mat = get_g_matrix(nds)
print(g_mat)

[[7 6 3 ... 6 5 5]
 [7 7 0 ... 3 3 3]
 [7 7 7 ... 1 1 7]
 ...
 [7 7 7 ... 7 6 2]
 [7 7 7 ... 7 7 4]
 [7 7 7 ... 7 7 7]]


#### g_mat -> networkx graph

In [11]:
import networkx as nx
G = nx.Graph()

In [12]:
G.add_nodes_from(range(nsize))

In [13]:
for i_idx in range(nsize):
    for j_idx in range(i_idx+1,nsize):
        if g_mat[i_idx][j_idx] != 7 :
            G.add_edge(i_idx,j_idx)

In [16]:
def gmat2graph(g_mat,nsize):
    G = nx.Graph()
    G.add_nodes_from(range(nsize))
    for i_idx in range(nsize):
        for j_idx in range(i_idx+1,nsize):
            if g_mat[i_idx][j_idx] != 7 :
                G.add_edge(i_idx,j_idx)
    
    return G

In [20]:
get_graph_info(gmat2graph(g_mat,nsize))

([Node(id=0, inputs=[], type=0),
  Node(id=1, inputs=[0], type=-1),
  Node(id=2, inputs=[0, 1], type=-1),
  Node(id=3, inputs=[1, 2], type=-1),
  Node(id=4, inputs=[0, 1, 2, 3], type=-1),
  Node(id=5, inputs=[0, 1, 2, 3, 4], type=-1),
  Node(id=6, inputs=[0, 1, 2, 3, 4, 5], type=-1),
  Node(id=7, inputs=[0, 1, 2, 3, 6], type=-1),
  Node(id=8, inputs=[0, 1, 2, 3, 4, 6, 7], type=-1),
  Node(id=9, inputs=[4, 5, 6, 8], type=-1),
  Node(id=10, inputs=[0, 3, 4, 5, 7, 8, 9], type=-1),
  Node(id=11, inputs=[1, 3, 4, 6, 7, 8, 9, 10], type=-1),
  Node(id=12, inputs=[0, 1, 3, 4, 5, 6, 10, 11], type=-1),
  Node(id=13, inputs=[1, 2, 7, 9, 10, 11, 12], type=-1),
  Node(id=14, inputs=[1, 5, 6, 7, 11, 13], type=-1),
  Node(id=15, inputs=[0, 1, 2, 3, 4, 5, 8, 9, 10, 12, 13, 14], type=-1),
  Node(id=16, inputs=[0, 1, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15], type=-1),
  Node(id=17, inputs=[0, 2, 3, 4, 5, 6, 9, 10, 11, 12, 14, 16], type=-1),
  Node(id=18, inputs=[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 1

In [21]:
nds

[Node(id=0, inputs=[], type=0),
 Node(id=1, inputs=[0], type=-1),
 Node(id=2, inputs=[0, 1], type=-1),
 Node(id=3, inputs=[1, 2], type=-1),
 Node(id=4, inputs=[0, 1, 2, 3], type=-1),
 Node(id=5, inputs=[0, 1, 2, 3, 4], type=-1),
 Node(id=6, inputs=[0, 1, 2, 3, 4, 5], type=-1),
 Node(id=7, inputs=[0, 1, 2, 3, 6], type=-1),
 Node(id=8, inputs=[0, 1, 2, 3, 4, 6, 7], type=-1),
 Node(id=9, inputs=[4, 5, 6, 8], type=-1),
 Node(id=10, inputs=[0, 3, 4, 5, 7, 8, 9], type=-1),
 Node(id=11, inputs=[1, 3, 4, 6, 7, 8, 9, 10], type=-1),
 Node(id=12, inputs=[0, 1, 3, 4, 5, 6, 10, 11], type=-1),
 Node(id=13, inputs=[1, 2, 7, 9, 10, 11, 12], type=-1),
 Node(id=14, inputs=[1, 5, 6, 7, 11, 13], type=-1),
 Node(id=15, inputs=[0, 1, 2, 3, 4, 5, 8, 9, 10, 12, 13, 14], type=-1),
 Node(id=16, inputs=[0, 1, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15], type=-1),
 Node(id=17, inputs=[0, 2, 3, 4, 5, 6, 9, 10, 11, 12, 14, 16], type=-1),
 Node(id=18, inputs=[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17], typ

#### Tripelet 적용

In [22]:
from utils_kyy.utils_hr import operation_dictionary

operation_dict=operation_dictionary()

In [23]:
operation_dict

{0: utils_kyy.utils_hr.conv2d_3x3,
 1: utils_kyy.utils_hr.depthwise_conv_3x3,
 2: utils_kyy.utils_hr.separable_conv_3x3,
 3: utils_kyy.utils_hr.depthwise_separable_conv_3x3,
 4: utils_kyy.utils_hr.maxpool2d_3x3,
 5: utils_kyy.utils_hr.avgpool2d_3x3,
 6: utils_kyy.utils_hr.identity}

In [24]:
import torch
import torch.nn as nn

import math

from utils_kyy.utils_graph import load_graph, get_graph_info



class depthwise_separable_conv_3x3(nn.Module):
    def __init__(self, nin, nout, stride):
        # input node 일때, stride = 1; => size 유지
        # input node 아닐 대, stride = 2; =>  (x-1)/2 + 1
        super(depthwise_separable_conv_3x3, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin, kernel_size=3, stride=stride, padding=1, groups=nin)
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1)  # default: stride=1, padding=0, dilation=1, groups=1, bias=True

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out
    

## new triplet_unint actually doulet (exclude conv)

##  double_unit (exclude conv in triplenet)
class double_unit(nn.Module):
    def __init__(self, inplanes, outplanes, stride=1):
        super(double_unit, self).__init__()
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm2d(outplanes)

    def forward(self, x):
        out = self.relu(x)
        out = self.bn(out)
        return out
    
    
class Triplet_unit(nn.Module):
    def __init__(self, inplanes, outplanes, stride=1):
        super(Triplet_unit, self).__init__()
        self.relu = nn.ReLU()
    
        self.conv = depthwise_separable_conv_3x3(inplanes, outplanes, stride)
        self.bn = nn.BatchNorm2d(outplanes)

    def forward(self, x):
        out = self.relu(x)
        out = self.conv(out)
        out = self.bn(out)
        return out

    
class Node_OP(nn.Module):
    def __init__(self, Node, inplanes, outplanes):
        super(Node_OP, self).__init__()
        self.is_input_node = Node.type == 0
        self.input_nums = len(Node.inputs)    # 해당 Node에 input으로 연결된 노드의 개수

        ## For hierachical representation
        self.id = Node.id
        self.oplist = nn.ModuleList()
        
        # get operation from g_matirx
        
        if self.input_nums > 1:
            for in_idx in Node.inputs:
                operation_idx = g_mat[in_idx][self.id]
                op = operation_dict[operation_idx]
                self.oplist.append(op(nin=inplanes,nout=outplanes, stride=1))

        
        # input 개수가 1보다 크면, 여러 input을 합쳐야함.
        if self.input_nums > 1:
            self.mean_weight = nn.Parameter(torch.ones(self.input_nums))  # type: torch.nn.parameter.Parameter
            self.sigmoid = nn.Sigmoid()

        if self.is_input_node:
            self.conv = Triplet_unit(inplanes, outplanes, stride=2)   # Triplet_unit 여기서 operation parameter 전달 G에서 꺼내서
        else:
            self.conv = double_unit(outplanes, outplanes, stride=1)

    # [참고] nn.Sigmoid()(torch.ones(1)) = 0.7311
    # seoungwonpark source 에서는 torch.zeros()로 들어감. => 0.5
    def forward(self, *input):
        if self.input_nums > 1:
            out = self.sigmoid(self.mean_weight[0]) * self.oplist[0](input[0]) ## input node가 아니라고 생각해도 됨!
            for i in range(1, self.input_nums):
                out = out + self.sigmoid(self.mean_weight[i]) *  self.oplist[i](input[i]) 
        else:
            out = input[0]
        out = self.conv(out)
        return out

    

class StageBlock(nn.Module):
    def __init__(self, graph, inplanes, outplanes):
        super(StageBlock, self).__init__()
        # graph를 input으로 받아서, Node_OP class. 즉, neural network graph로 전환함.
        self.nodes, self.input_nodes, self.output_nodes = get_graph_info(graph)
        self.nodeop  = nn.ModuleList()    # Holds submodules in a list.
        for node in self.nodes:
            # 각각의 node들을 Node_OP class로 만들어준 뒤, nn.ModuleList()인 self.nodeop에 append 해주기
            self.nodeop.append(Node_OP(node, inplanes, outplanes))

    def forward(self, x):
        results = {}
        # input
        for id in self.input_nodes:
            results[id] = self.nodeop[id](x)  # input x를 먼저 graph's input node에 각각 넣어줌.

        # graph 중간 계산
        for id, node in enumerate(self.nodes):
            # 각각의 노드 id에 대해
            if id not in self.input_nodes:
                # graph's input node가 아니라면, 그래프 내에서 해당 노드의 인풋들인 node.inputs의 output인 results[_id]
                #    => 그 결과를 results[id]에 저장.
                # self.nodeop[id]는 해당 id의 Node_OP. 즉, input들을 받아서 forward(모아서, conv 태우기)하는 것.
                # 따라서, input으로 넣을 때 unpack 함.
                # id 작은 노드부터 result를 차근차근 계산하면서, id를 올라감.
                results[id] = self.nodeop[id](*[results[_id] for _id in node.inputs])

        result = results[self.output_nodes[0]]
        # output
        # graph's output_nodes의 output 들을 평균내기
        for idx, id in enumerate(self.output_nodes):
            if idx > 0:
                result = result + results[id]
        result = result / len(self.output_nodes)
        return result
   
 
class RWNN(nn.Module):
    def __init__(self, net_type, graphs, channels, num_classes=1000, input_channel=3):
        super(RWNN, self).__init__()

        self.input_channel = input_channel
        # 논문에서도 conv1 쪽은 예외적으로 Conv-BN 이라고 언급함. (나머지에서는 Conv-ReLU-BN 을 conv 로 표기) 
#         self.conv1 = depthwise_separable_conv_3x3(input_channel, channels // 2, 2)    # nin, nout, stride
        self.conv1 = depthwise_separable_conv_3x3(input_channel, channels // 2, 1)    # nin, nout, stride        
        self.bn1 = nn.BatchNorm2d(channels // 2)
        
        # 채널수 변화도, 논문에서처럼 conv2: C, conv3: 2C, conv4: 4C, conv5: 8C
        if net_type == 'small':
#             self.conv2 = Triplet_unit(channels // 2, channels, 2)    # inplanes, outplanes, stride=2
            self.conv2 = Triplet_unit(channels // 2, channels, 1)    # inplanes, outplanes, stride=1            

#             self.conv3 = StageBlock(graphs.stage_1, channels // 2, channels)
            self.conv3 = StageBlock(graphs.stage_1, channels, channels)        
 
            self.conv4 = StageBlock(graphs.stage_2, channels, channels * 2)   

            self.conv5 = StageBlock(graphs.stage_3, channels * 2, channels * 4)

            self.relu = nn.ReLU()
            
            self.bn2 = nn.BatchNorm2d(channels * 4)
            self.avgpool = nn.AvgPool2d(4, stride=1)  # 마지막은 global average pooling
            self.fc = nn.Linear(channels * 4, num_classes)

#         if net_type == 'small':
#             self.conv2 = Triplet_unit(channels // 2, channels, 2)    # inplanes, outplanes, stride=2

#             self.conv3 = StageBlock(graphs.stage_1, channels, channels)
 
#             self.conv4 = StageBlock(graphs.stage_2, channels, channels *2)   

#             self.conv5 = StageBlock(graphs.stage_3, channels * 2, channels * 4)

#             self.relu = nn.ReLU()
# #             self.conv = nn.Conv2d(channels * 4, 1280, kernel_size=1)   # 마지막에 1x1 conv, 1280-d
# #             self.bn2 = nn.BatchNorm2d(1280)
        
        #######################################
        # 원 코드에서 regular 부분 지움
        #######################################
#         self.avgpool = nn.AvgPool2d(7, stride=1)  # 마지막은 global average pooling
#         self.fc = nn.Linear(1280, num_classes)
    
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
        
        
    def forward(self, x):
#         print("==========Data Size", x.size())
        x = self.conv1(x)
        x = self.bn1(x)
#         print("==========Conv1 Size", x.size())  # => (-, 54, 16, 16)

        x = self.conv2(x)
#         print("==========Conv2  Size", x.size())  # => (-, 109, 8, 8)

        x = self.conv3(x)
#         print("==========Conv3  Size", x.size())  # => (-, 109, 8, 8)

        x = self.conv4(x)
#         print("==========Conv4  Size", x.size())  # => (-, 218, 4, 4)

        x = self.conv5(x)
#         print("==========Conv5  Size", x.size())  # => (-, 436, 2, 2)

#         x = self.relu(x)  # => (-, 436, 1, 1)
#         x = self.conv(x)
#         print("==========Conv6  Size", x.size())  # => (-, 1280, 1, 1)

        x = self.bn2(x)
        x = self.relu(x)

#         print("==========before avgpool  Size", x.size())   ## [수정] CIFAR-10 에서는 여기까지오면 (-, 1280, 7, 7) 이 아니라 (-, ,1280, 1, 1)
        x = self.avgpool(x)
#         print("==========after avgpool  Size", x.size())
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

### Check
* 임의로 graphs 만들어서 stage block -> RWNN 구성해보기
* torch viz로 확인

In [25]:
from easydict import EasyDict


In [26]:
len(nds)

32

In [27]:
StageBlock(grph_ex, 20, 20)


StageBlock(
  (nodeop): ModuleList(
    (0): Node_OP(
      (oplist): ModuleList()
      (conv): Triplet_unit(
        (relu): ReLU()
        (conv): depthwise_separable_conv_3x3(
          (depthwise): Conv2d(20, 20, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=20)
          (pointwise): Conv2d(20, 20, kernel_size=(1, 1), stride=(1, 1))
        )
        (bn): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Node_OP(
      (oplist): ModuleList()
      (conv): double_unit(
        (relu): ReLU()
        (bn): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): Node_OP(
      (oplist): ModuleList(
        (0): depthwise_separable_conv_3x3(
          (depthwise): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=20)
          (pointwise): Conv2d(20, 20, kernel_size=(1, 1), stride=(1, 1))
        )
        (1): conv2d_3x3(
          (conv): Conv2d(20

In [29]:
## 37번 노드 확인해보면

# input node 0,1,31,35,36
get_graph_info(grph_ex)[0][20]

# operation은 depthwise_seperable, depthwi, conv2d, conv2d, maxpooling 

Node(id=20, inputs=[0, 1, 2, 3, 4, 6, 7, 8, 9, 11, 13, 14, 16, 17, 18], type=-1)

In [30]:
pprint.pprint(g_mat[:,20])

array([0, 4, 0, 6, 6, 7, 3, 3, 6, 6, 7, 5, 7, 2, 5, 7, 1, 2, 0, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7])


In [85]:
operation_dict

{0: utils_kyy.utils_hr.conv2d_3x3,
 1: utils_kyy.utils_hr.depthwise_conv_3x3,
 2: utils_kyy.utils_hr.separable_conv_3x3,
 3: utils_kyy.utils_hr.depthwise_separable_conv_3x3,
 4: utils_kyy.utils_hr.maxpool2d_3x3,
 5: utils_kyy.utils_hr.avgpool2d_3x3,
 6: utils_kyy.utils_hr.identity}

* RWNN 에서 Check

In [31]:
graphs= EasyDict({
                 'stage_1':grph_ex,
                 'stage_2':grph_ex,
                 'stage_3':grph_ex})

In [32]:
RWNN(net_type='small', graphs=graphs, channels=109, num_classes=10, input_channel=3)


RWNN(
  (conv1): depthwise_separable_conv_3x3(
    (depthwise): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
    (pointwise): Conv2d(3, 54, kernel_size=(1, 1), stride=(1, 1))
  )
  (bn1): BatchNorm2d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Triplet_unit(
    (relu): ReLU()
    (conv): depthwise_separable_conv_3x3(
      (depthwise): Conv2d(54, 54, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=54)
      (pointwise): Conv2d(54, 109, kernel_size=(1, 1), stride=(1, 1))
    )
    (bn): BatchNorm2d(109, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): StageBlock(
    (nodeop): ModuleList(
      (0): Node_OP(
        (oplist): ModuleList()
        (conv): Triplet_unit(
          (relu): ReLU()
          (conv): depthwise_separable_conv_3x3(
            (depthwise): Conv2d(109, 109, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=109)
            (pointwise): Conv2d(109, 109, 